In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.052475929260253906
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 11.791639358292162
200000 21.47830217050178
300000 26.26850571117157
400000 25.00922266968624
500000 27.713393197422945
600000 28.481038294070363
700000 32.77033515453877
800000 37.613283291254824
900000 41.87749462894796
1000000 49.21467487864677
1100000 51.50152217092535
1200000 36.279622188062866
1300000 26.546573842042758
1400000 48.2156685281176
1500000 48.312059593985154
1600000 48.370406973305045
1700000 48.997966468086084
1800000 50.870933584463394
1900000 52.253290521513435
2000000 48.23379597199117
2100000 48.14928301325114
2200000 53.28448045751211
2300000 54.6778796556676
2400000 39.304803077117604
2500000 19.86760057752856
2600000 46.33686485524749
2700000 45.841640529378786
2800000 45.31064740612307
2900000 44.75075539703649
3000000 45.28697

29700000 49.872245939583465
29800000 49.64496262806008
29900000 49.01023698073984
30000000 46.2300533117679
30100000 23.634030309789292
30200000 38.21816534412708
30300000 46.61557784234566
30400000 48.393478362887315
30500000 50.96182719424391
30600000 61.48284574470397
30700000 48.93671316831979
30800000 49.76428613409628
30900000 49.99463935735585
31000000 50.02546876779758
31100000 23.88262417756505
31200000 46.693446918677324
31300000 51.94656710621654
31400000 50.889883588187026
31500000 55.50544581503326
31600000 33.422931259965104
31700000 51.82374536544757
31800000 49.78966184150977
31900000 49.4119703972687
32000000 49.061825485960085
32100000 49.67764653139213
32200000 47.88951026104295
32300000 47.42014557595922
32400000 44.86418936944579
32500000 26.47971337483098
32600000 34.79921019625182
32700000 44.46728884163727
32800000 46.2039487765502
32900000 46.816458878328575
33000000 48.18969992235137
33100000 46.95409413992505
33200000 49.23660268306733
33300000 49.37356695823

59800000 23.1629090963539
59900000 47.54540278186536
60000000 46.98894933730382
60100000 45.642702657175164
60200000 49.97544801802421
60300000 58.9786283600025
60400000 52.68780463916025
60500000 47.89448374041738
60600000 37.01930405001579
60700000 27.861943025088557
60800000 45.383276040482784
60900000 44.598142391524824
61000000 44.79230546367868
61100000 43.83319945792954
61200000 42.77453384408554
61300000 41.52999734234267
61400000 14.101732554792298
61500000 41.665229567441436
61600000 42.35513627404531
61700000 42.83544228238388
61800000 42.005184935442976
61900000 45.114924076507016
62000000 44.72621874051991
62100000 38.274525443672275
62200000 30.193026565117805
62300000 49.40014981381566
62400000 57.16982307393212
62500000 48.301644224221825
62600000 47.849853384548815
62700000 48.29845362651742
62800000 48.25482431048271
62900000 48.60874229410685
63000000 45.744220040598194
63100000 19.712741796301604
63200000 46.05034800807249
63300000 45.790578072724145
63400000 43.652

89900000 45.34635982352517
90000000 43.844097499272515
90100000 38.229361820258376
90200000 38.66639076008621
90300000 12.940007547501478
90400000 39.73162692100985
90500000 39.76187011871427
90600000 37.9802049173323
90700000 39.35169421254284
90800000 20.873033916431545
90900000 36.01304736697326
91000000 41.607421964928314
91100000 40.71322847016907
91200000 44.04579482570167
91300000 43.09570756676765
91400000 42.366367045863726
91500000 44.05975333532705
91600000 45.340629621893434
91700000 44.046079635001654
91800000 44.70448064519717
91900000 42.82875225601821
92000000 43.01910134568394
92100000 35.68305135510449
92200000 20.30846373897565
92300000 40.608744963032585
92400000 38.44100552541304
92500000 40.84138646239698
92600000 41.733202466119735
92700000 9.704680426005476
92800000 42.089520332351725
92900000 39.62862907534746
93000000 40.041494069370174
93100000 39.44509335392701
93200000 18.591074068482705
93300000 38.833004303160145
93400000 42.94221917840768
93500000 49.468

119200000 21.952576749946513
119300000 35.28789750828081
119400000 34.89112221539701
119500000 18.85664789602882
119600000 36.62644294265377
119700000 36.7142265541759
119800000 35.738722399332545
119900000 38.39808203468171
120000000 39.87318240727296
120100000 39.90464781670253
120200000 40.95376226072246
120300000 40.60408460035889
120400000 40.86228191966364
120500000 43.174750577300706
120600000 41.44251844955728
120700000 40.49592607821955
120800000 38.71525340870391
120900000 39.5304971830766
121000000 37.396195376840595
121100000 38.58116540756978
121200000 28.863917828454195
121300000 22.934393435306877
121400000 36.627079038090926
121500000 28.77266265024819
121600000 19.94308749501027
121700000 36.40357474144564
121800000 34.93736860575748
121900000 27.69936665213665
122000000 48.974177904941804
122100000 37.98332592901583
122200000 37.036661512592296
122300000 38.88629681370777
122400000 39.530276017767676
122500000 41.91705351779347
122600000 42.033088378709714
122700000 4

...

523100000 5.021960621735109
523200000 4.685453480841007
523300000 4.95696809541663
523400000 5.2593807141160545
523500000 4.945364944049123
523600000 4.811367781869061
523700000 4.720343019761878
523800000 4.997224286821551
523900000 3.87100848681057
524000000 2.5477325976650973
524100000 3.149643159375508
524200000 4.744998339283414
524300000 4.835933149755514
524400000 4.6439806061534785
524500000 4.946354518129651
524600000 4.912077460335907
524700000 4.785407485632522
524800000 4.623824037949232
524900000 4.470136429317352
525000000 4.836682683782682
525100000 3.810707951820002
525200000 2.6345324568483175
525300000 3.2017009191203756
525400000 4.654664099074742
525500000 4.503781119380136
525600000 4.585694431859263
525700000 4.696964570351998
525800000 4.9846997358110565
525900000 4.918929604074698
526000000 4.7782961134337905
526100000 4.920985428383738
526200000 4.607612951355227
526300000 3.8319384804100385
526400000 1.8676018095906495
526500000 3.7452284417283557
526600000 4.

552000000 3.304129816063002
552100000 3.9595932844725916
552200000 4.142149081576056
552300000 4.1843030919285775
552400000 3.7867921283259522
552500000 2.882010641134723
552600000 3.233159761872758
552700000 3.6001033328289602
552800000 3.1516432320351107
552900000 2.428439840431963
553000000 3.5014030399162426
553100000 3.6656533135039036
553200000 2.8722423932104166
553300000 3.9314139608135394
553400000 4.042284144989894
553500000 4.000995528854762
553600000 3.6850022470050487
553700000 2.5309621869616845
553800000 3.5672685768871566
553900000 3.555387430168273
554000000 3.199341010157728
554100000 2.286115436615487
554200000 3.403611328040205
554300000 3.530470945323434
554400000 2.89559836378226
554500000 3.8149692498743324
554600000 3.917391349571731
554700000 3.859339062927815
554800000 3.8305333084642212
554900000 2.277132743355471
555000000 3.7319084503429623
555100000 3.592024560021736
555200000 3.102650595344746
555300000 2.1141423721479393
555400000 3.348492379381194
55550

579500000 2.1291190361326815
579600000 2.117940289394718
579700000 1.758981380313949
579800000 1.960723806293585
579900000 1.9713003022613975
580000000 2.263884453637677
580100000 2.3248236341104507
580200000 1.9471012833835808
580300000 1.7881937262255596
580400000 1.8863641540599942
580500000 1.8992723577470267
580600000 2.416316428777434
580700000 2.53230716500867
580800000 1.7897255517081163
580900000 1.8115358941669013
581000000 1.873135793629676
581100000 1.7550948139062608
581200000 2.508222830311859
581300000 2.510003713805296
581400000 1.67393058233975
581500000 1.8134556263878359
581600000 1.7934353402110697
581700000 1.6944725357311896
581800000 2.441348744410185
581900000 2.448697319831747
582000000 1.6499032306744237
582100000 1.8781484498535532
582200000 1.8087202159242473
582300000 1.6788890206881932
582400000 2.415935563289167
582500000 2.4886968525810835
582600000 1.666972566237094
582700000 1.8435212252819284
582800000 1.989319605547297
582900000 2.165451748398142
583

608000000 0.869511511042136
608100000 1.1252394490894293
608200000 0.988341212681669
608300000 1.1384415379349786
608400000 1.0419553024337114
608500000 1.1716593315431092
608600000 0.7161251148700197
608700000 1.1188897606446075
608800000 1.1648604990170366
608900000 1.044120712247318
609000000 0.8943615907516949
609100000 1.1283028974959348
609200000 0.7368969317154189
609300000 1.2262128888241794
609400000 1.0898735201346352
609500000 0.9132269395198226
609600000 0.9271554176696553
609700000 1.1842498011209983
609800000 0.6810144054791927
609900000 1.1012510010050185
610000000 1.2076331870116048
610100000 0.8449163429206747
610200000 0.877513225529125
610300000 1.083427246182922
610400000 0.6752250476182202
610500000 1.192604131263995
610600000 1.069775070952255
610700000 0.7197924957930445
610800000 0.8997534431313489
610900000 1.2014286842246673
611000000 0.677189323246783
611100000 1.0697862395967013
611200000 1.0957410850855385
611300000 0.7735895596838395
611400000 0.9250932087

634100000 0.00018746459806561472
best so far: 0
type: [2, 1, 67, 1, 1] 134
cases of this type: 4489
best so far: 0
type: [2, 67, 1, 1, 1] 134
cases of this type: 67
best so far: 0
type: [67, 1, 1, 1, 2] 134
cases of this type: 16
best so far: 0
type: [67, 1, 1, 2, 1] 134
cases of this type: 8
best so far: 0
type: [67, 1, 2, 1, 1] 134
cases of this type: 4
best so far: 0
type: [67, 2, 1, 1, 1] 134
cases of this type: 2
best so far: 0
type: [134, 1, 1, 1, 1] 134
cases of this type: 1
best so far: 0
all cases: 634149671
best: 0
